In [222]:
from sklearn import datasets
import pandas as pd
import math
iris = datasets.load_iris()

In [223]:
df1=pd.DataFrame(iris.data)
df2=pd.DataFrame(iris.target)
df=pd.concat([df1,df2],axis=1)
df.head()

,0,1,2,3,0
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [224]:
df.columns = ["sl", "sw", 'pl', 'pw','Label']

In [225]:
df.head()

,sl,sw,pl,pw,Label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [226]:
def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [227]:
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')


In [228]:
df.head()

,sl,sw,pl,pw,Label,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,0,b,c,a,a
1,4.9,3.0,1.4,0.2,0,a,b,a,a
2,4.7,3.2,1.3,0.2,0,a,c,a,a
3,4.6,3.1,1.5,0.2,0,a,c,a,a
4,5.0,3.6,1.4,0.2,0,a,c,a,a


In [229]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [230]:
df.head()

,Label,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,0,b,c,a,a
1,0,a,b,a,a
2,0,a,c,a,a
3,0,a,c,a,a
4,0,a,c,a,a


In [253]:
df['Label'].count()

150

In [254]:
unused_features = df.columns[1:5]
unused_features = list(unused_features)
print(unused_features)

['sl_labeled', 'sw_labeled', 'pl_labeled', 'pw_labeled']


In [263]:
def entropy(s):
    e=0
    for val in s.value_counts():
        p=val/s.count()
        e+= -1*p*(math.log(p,2))
    return e


In [284]:
def buildtree(df,unused_features,level):
    l=[]        
    if(unused_features!=[] and entropy(df['Label'])!=0):

        for f in unused_features:
            possible_vals=set(df[f])
            comb_entropy=0
            for val in possible_vals:
                s=df[df[f]==val]['Label']
                comb_entropy+=entropy(s)*(s.count()/df['Label'].count())
            gain=entropy(df['Label'])-comb_entropy
            l.append(gain)
    
        maxgain=max(l)
        best_feature = unused_features[l.index(maxgain)]
        print("Level",level)
        print("Current Entropy is",entropy(df['Label']))
        values=df['Label'].value_counts().keys().tolist()
        for val in values:
            print("Count of ",val,"is",df['Label'].value_counts(ascending=True)[val])
        print("Splitting on feature",best_feature,"with gain",maxgain)
        print()
        newunused_features=unused_features.copy()
        newunused_features.remove(best_feature)
    
    
        possible_vals=set(df[best_feature])
        for val in possible_vals:
            buildtree(df[df[best_feature]==val],newunused_features,level+1)
    
    else:
        print("Level",level)
        print("Current Entropy is",entropy(df['Label']))
        values=df['Label'].value_counts().keys().tolist()
        for val in values:
            print("Count of" ,val,"is",df['Label'].value_counts(ascending=True)[val])
        print("Reached leaf node")
        print()
        return


In [285]:
entropy(df['Label'])

1.584962500721156

In [286]:
buildtree(df,unused_features,0)

Level 0
Current Entropy is 1.584962500721156
Count of  2 is 50
Count of  1 is 50
Count of  0 is 50
Splitting on feature pw_labeled with gain 1.2627308217896138

Level 1
Current Entropy is 0.0
Count of 0 is 50
Reached leaf node

Level 1
Current Entropy is 0.863120568566631
Count of  1 is 40
Count of  2 is 16
Splitting on feature pl_labeled with gain 0.3107100419019655

Level 2
Current Entropy is 0.0
Count of 1 is 1
Reached leaf node

Level 2
Current Entropy is 0.6581912658132185
Count of  1 is 39
Count of  2 is 8
Splitting on feature sl_labeled with gain 0.15790810784426046

Level 3
Current Entropy is 0.0
Count of 1 is 14
Reached leaf node

Level 3
Current Entropy is 0.0
Count of 1 is 2
Reached leaf node

Level 3
Current Entropy is 0.783776947484701
Count of  1 is 23
Count of  2 is 7
Splitting on feature sw_labeled with gain 0.08807926473635475

Level 4
Current Entropy is 0.8812908992306927
Count of 1 is 14
Count of 2 is 6
Reached leaf node

Level 4
Current Entropy is 0.0
Count of 1 is 